In [2]:
# Config:
# Feature - real, imaginary, magnitude
# Sample - 100 CIRs
# Non-zero coefficients - 10
# -------------------------------------------------------------------------------------------------------
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:100, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:100, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude))

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude))


# Atoms for our dictionary - Reshape Alice's data to flatten each sample.
alice_atoms = alice_features.reshape(100, -1)  # Each sample becomes a long vector.
eve_atoms = eve_features.reshape(100, -1)
# Stack the data for both classes to form the full dictionary D
atoms = np.vstack((alice_atoms, eve_atoms))  # Combined data.


# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_atoms.shape[0])  # Label '0' for Alice.
eve_labels = np.ones(eve_atoms.shape[0])       # Label '1' for Eve.
# Combine labels.
true_label = np.hstack((alice_labels, eve_labels))  # Combined labels.

# Split the data into training and testing sets.
# Let's use 70% of the data for training and 30% for testing.
train_data, test_data, t_train_label, t_test_label = train_test_split(atoms, true_label, test_size=0.2, random_state=42, stratify=true_label)

# Build the dictionary using the training data.
dictionary = train_data.T  # Transpose so that each column is an atom.

# Now, we will classify each test sample and keep track of predictions.
predictions = []




[0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]


In [7]:


# Loop over each test signal.
for idx, test_signal in enumerate(test_data):
    test_signal = test_signal.reshape(-1)  # Ensure it's a column vector.

    # Create the OMP model.
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=10, tol=1e-1)

    # Fit the model to find the sparse coefficients.
    omp.fit(dictionary, test_signal)
    coefficients = omp.coef_

    # Initialize residuals list for this test signal.
    residuals = []

    # We have two classes: Alice (0) and Eve (1).
    for i in range(2):
        # Create an array of zeros like the coefficients.
        coef_class = np.zeros_like(coefficients)
        if i == 0:
            # For Alice, keep coefficients corresponding to Alice's atoms.
            alice_indices = np.where(t_train_label == 0)[0]
            coef_class[alice_indices] = coefficients[alice_indices]
        else:
            # For Eve, keep coefficients corresponding to Eve's atoms.
            eve_indices = np.where(t_train_label == 1)[0]
            coef_class[eve_indices] = coefficients[eve_indices]
        
        # Reconstruct the signal using only the coefficients from one class.
        reconstructed_signal = dictionary @ coef_class
        # Calculate the residual.
        residual = np.linalg.norm(test_signal - reconstructed_signal)
        # Add the residual to the list.
        residuals.append(residual)

    # Predict the class with the smallest residual.
    predicted_class = np.argmin(residuals)
    predictions.append(predicted_class)



# Calculate the accuracy.
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(t_test_label, predictions)
print(f"\nClassification Accuracy: {accuracy * 100:.2f}%")

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matchin


Classification Accuracy: 82.50%


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matchin